<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/Sarde/simple_two_layer_bidirectional_lstm_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple two-layer bidirectional LSTM with Pytorch

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import time
import copy
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
from torch.optim.lr_scheduler import ReduceLROnPlateau

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from spacy.lang.en import English
import string
import spacy
from torch.utils.data import Dataset, DataLoader
from transformers import BertPreTrainedModel, BertModel
from transformers import AutoConfig, AutoTokenizer
import torch

from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification,AutoTokenizer

from torch import nn

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import DebertaModel
from transformers import DebertaPreTrainedModel
from tqdm import tqdm, trange
import os
import gc
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import torchtext
import nltk
import time
from datetime import timedelta
import numpy as np
from sklearn import metrics
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.utils.data_utils import GeneratorEnqueuer  # We only want this for multithreaded

from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence
from torch import Tensor

## Define hyperparameters

In [3]:
n_epochs = 250
lr = 0.01
n_folds = 5
lstm_input_size = 1
hidden_state_size = 30
batch_size = 30
num_sequence_layers = 2
output_dim = 11
num_time_steps = 4000
rnn_type = 'LSTM'

## Define model

In [4]:
class Bi_RNN(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=11, num_layers=2, rnn_type='LSTM'):
        super(Bi_RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        #Define the initial linear hidden layer
        self.init_linear = nn.Linear(self.input_dim, self.input_dim)

        # Define the LSTM layer
        self.lstm = eval('nn.' + rnn_type)(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim * 2, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred

## Define data loaders

In [5]:
class CommonLitDataset(Dataset):
    #TODO: aggiungere max len specifica per tutti e 3 i campi
    def __init__(self, data, maxlen, tokenizer, input_cols, target_cols, split=True, padding=True):
        #Store the contents of the file in a pandas dataframe
        self.df = data.reset_index()
        #Initialize the tokenizer for the desired transformer model
        self.tokenizer = tokenizer
        #Maximum length of the tokens list to keep all the sequences of fixed size
        self.maxlen = maxlen
        #list of input columns
        self.input_cols = input_cols
        #list of target columns
        self.target_cols = target_cols
        self.prompt_text=[]
        self.prompt_question=[]
        self.summary=[]
        self.split=split
        self.padding=padding


    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        #Select the sentence and label at the specified index in the data frame
        prompt_text_tokens=[]
        prompt_question_tokens=[]
        summary_tokens=[]
        #["text","prompt_question","prompt_text"]

        if(self.split==True):
            prompt_text_tokens=['[CLS]'] + self.tokenizer.tokenize(self.df.loc[index, self.input_cols[2]])
            prompt_question_tokens=['[CLS]'] + self.tokenizer.tokenize(self.df.loc[index, self.input_cols[1]])
            summary_tokens=['[CLS]'] + self.tokenizer.tokenize(self.df.loc[index, self.input_cols[0]])
            if len(prompt_text_tokens) < self.maxlen:
                if self.padding:
                    prompt_text_tokens = prompt_text_tokens + ['[PAD]' for _ in range(self.maxlen - len(prompt_text_tokens))]+ ['[SEP]']
                else: prompt_text_tokens = prompt_text_tokens + ['[SEP]']
            else:
                prompt_text_tokens = prompt_text_tokens[:self.maxlen-1] + ['[SEP]']
            if len(prompt_question_tokens) < self.maxlen:
                if self.padding:
                    prompt_question_tokens = prompt_question_tokens + ['[PAD]' for _ in range(self.maxlen - len(prompt_question_tokens))]+ ['[SEP]']
                else: prompt_question_tokens = prompt_question_tokens + ['[SEP]']
            else:
                prompt_question_tokens = prompt_question_tokens[:self.maxlen-1] + ['[SEP]']
            if len(summary_tokens) < self.maxlen:
                if self.padding:
                    summary_tokens = summary_tokens + ['[PAD]' for _ in range(self.maxlen - len(summary_tokens))]+ ['[SEP]']
                else: summary_tokens = summary_tokens + ['[SEP]']
            else:
                summary_tokens = summary_tokens[:self.maxlen-1] + ['[SEP]']

            #Obtain the indices of the tokens in the BERT Vocabulary
            prompt_text_input_ids = self.tokenizer.convert_tokens_to_ids(prompt_text_tokens)
            prompt_question_tokens_input_ids = self.tokenizer.convert_tokens_to_ids(prompt_question_tokens)
            summary_tokens_input_ids = self.tokenizer.convert_tokens_to_ids(summary_tokens)

            prompt_text_input_ids = torch.tensor(prompt_text_input_ids)
            prompt_question_tokens_input_ids = torch.tensor(prompt_question_tokens_input_ids)
            summary_tokens_input_ids = torch.tensor(summary_tokens_input_ids)
            input_ids=[prompt_text_input_ids, prompt_question_tokens_input_ids, summary_tokens_input_ids]
            input_length=len(input_ids)

        else:
            #Select the sentence and label at the specified index in the data frame
            tokens=[]
            for col in self.input_cols:
              temp_tokens = self.tokenizer.tokenize(self.df.loc[index, col])
              tokens = tokens + temp_tokens + ['[SEP]']
            #Preprocess the text to be suitable for the transformer
            tokens = ['[CLS]'] + tokens
            if len(tokens) < self.maxlen:
                if self.padding:
                  input_length=len(tokens)
                  tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))]
            else:
                tokens = tokens[:self.maxlen-1]

            #Obtain the indices of the tokens in the BERT Vocabulary
            input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            input_ids = torch.tensor(input_ids)
        try:
            target = self.df.loc[index, self.target_cols]
        except Exception as e:
            raise e
        target = torch.tensor(target, dtype=torch.float32)
        return input_ids, input_length, target

In [6]:
print(os.getcwd())
summaries_train_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/summaries_train.csv"
prompt_train_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/prompts_train.csv"

summaries_test_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/summaries_test.csv"
prompt_test_path = "https://raw.githubusercontent.com/Sopralapanca/CommonLitChallenge/main/data/prompts_test.csv"

train_data = pd.read_csv(summaries_train_path, sep=',', index_col=0)
prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)

/content


In [7]:
training_data = train_data.merge(prompt_data, on='prompt_id')

In [8]:
training_data

,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
1,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
2,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
3,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
4,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
...,...,...,...,...,...,...,...
7160,39c16e,"It has to be made on a complex storyline, with...",-0.981265,-1.548900,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \nAs the sequel to what has already...
7161,39c16e,Aristotle descirbes an ideal tradgedy as being...,-0.511077,-1.589115,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \nAs the sequel to what has already...
7162,39c16e,A tragedy should have a complex plan not a sim...,-0.834946,-0.593749,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \nAs the sequel to what has already...
7163,39c16e,Aristotle believed that the ideal tradegy shou...,-0.157460,-0.165811,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \nAs the sequel to what has already...


In [9]:
training_data['prompt_text_length']= training_data['prompt_text'].str.len()
training_data['prompt_question_length']=training_data['prompt_question'].str.len()
training_data['student_summary_length']=training_data['text'].str.len()

In [10]:
train, test = train_test_split(training_data,  test_size=0.2, shuffle=True)
train, validation = train_test_split(train,  test_size=0.2, shuffle=True)

In [11]:
training_data.describe()

,content,wording,prompt_text_length,prompt_question_length,student_summary_length
count,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000
mean,-0.014853,-0.063072,3875.733845,118.833775,418.776971
std,1.043569,1.036048,784.383272,42.239220,307.833685
min,-1.729859,-1.962614,3329.000000,77.000000,114.000000
25%,-0.799545,-0.872720,3329.000000,77.000000,216.000000
50%,-0.093814,-0.081769,3360.000000,104.000000,320.000000
75%,0.499660,0.503833,5132.000000,184.000000,513.000000
max,3.900326,4.310693,5132.000000,184.000000,3940.000000


In [12]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [13]:
MAX_LEN=2000
BATCH_SIZE=10
split=False
padding= True
input_cols=["text","prompt_question","prompt_text"]
target_cols=["content", "wording"]
train_set = CommonLitDataset(data=train, maxlen=MAX_LEN, tokenizer=tokenizer, input_cols=input_cols, target_cols=target_cols, split=split, padding=padding)
valid_set = CommonLitDataset(data=validation, maxlen=MAX_LEN, tokenizer=tokenizer, input_cols=input_cols, target_cols=target_cols, split=split, padding=padding)
test_set = CommonLitDataset(data=test, maxlen=MAX_LEN, tokenizer=tokenizer, input_cols=input_cols, target_cols=target_cols, split=split, padding=padding)

train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

In [14]:
sample = next(iter(train_loader))

In [15]:
sample


[tensor([[  101,  1789,  1104,  ...,     0,     0,     0],
         [  101,  1109,  1503,  ...,     0,     0,     0],
         [  101,   157, 20240,  ...,     0,     0,     0],
         ...,
         [  101,  1370,  1122,  ...,     0,     0,     0],
         [  101,  1109,  2890,  ...,     0,     0,     0],
         [  101,  1109,  4180,  ...,     0,     0,     0]]),
 tensor([1262,  830,  853, 1316,  805, 1290, 1386, 1552,  793,  782]),
 tensor([[-1.5472, -1.4612],
         [ 1.1083,  1.6282],
         [-0.7147, -0.9129],
         [ 0.5591, -0.6349],
         [-1.5472, -1.4612],
         [-0.6119, -0.1629],
         [ 1.1138,  0.4169],
         [-0.3823, -1.7955],
         [-0.1732, -0.1285],
         [ 1.0487,  1.2929]])]

In [70]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, output_dim, n_layers,
                 bidirectional, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim1,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            batch_first=True)
        self.fc1 = nn.Linear(hidden_dim1 * 2, hidden_dim2)
        self.fc2 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        packed_embedded = pack_padded_sequence(embedded, text_lengths.to('cpu'), batch_first=True, enforce_sorted=False).to(device)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        rel = self.relu(cat)
        dense1 = self.fc1(rel)
        drop = self.dropout(dense1)
        preds = self.fc2(drop)
        return preds


In [71]:
def evaluate(model, criterion, dataloader, device):
    model.eval()
    mean_acc, mean_loss, count = 0, 0, 0

    with torch.no_grad():
        for input_ids, text_length, target in (dataloader):

            input_ids, text_length, target = input_ids.to(device), text_length.to(device), target.to(device)
            output = model(input_ids, text_length)

            mean_loss += criterion(output, target).item()
            #mean_loss += get_rmse(output, target.type_as(output)).item()
            count += 1

    return mean_loss/count


from tqdm import tqdm, trange

def train(model, criterion, optimizer, train_loader, val_loader, epochs, device):
    best_acc = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for (input_ids, input_length, target) in tqdm(train_loader):
            optimizer.zero_grad()

            input_ids, target = input_ids.to(device), target.to(device)
            input_length= input_length.to(device)
            output = model(input_ids, input_length)

            loss = criterion(output, target.type_as(output))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        tl = train_loss/len(train_loader)
        val_loss = evaluate(model=model, criterion = criterion, dataloader=val_loader, device=device)
        print(f"Epoch {epoch} - Training Loss: {tl} Validation Loss: {val_loss}")

In [72]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss


class MCRMSELoss(nn.Module):
    def __init__(self, num_scored=2):
        super().__init__()
        self.rmse = RMSELoss()
        self.num_scored = num_scored

    def forward(self, yhat, y):
        score = 0
        for i in range(self.num_scored):
            score += self.rmse(yhat[:, i], y[:, i]) / self.num_scored

        return score

In [73]:
LR = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTM(vocab_size=30522, embedding_dim=20, bidirectional=True,dropout=0.5,hidden_dim1=100, hidden_dim2=50,n_layers=2,output_dim=2)
model.to(device)
OPTIMIZER = optim.Adam(params=model.parameters(), lr=LR)
EPOCHS = 5
CRITERION = MCRMSELoss()
train(model=model,
      criterion=CRITERION,
      optimizer=OPTIMIZER,
      train_loader=train_loader,
      val_loader=valid_loader,
      epochs = EPOCHS,
     device = device)

100%|██████████| 459/459 [02:39<00:00,  2.87it/s]


Epoch 0 - Training Loss: 0.9739451816139139 Validation Loss: 0.9206880823425625


100%|██████████| 459/459 [02:34<00:00,  2.97it/s]


Epoch 1 - Training Loss: 0.936924766221597 Validation Loss: 0.9076655843983525


100%|██████████| 459/459 [02:34<00:00,  2.98it/s]


Epoch 2 - Training Loss: 0.9085229719646096 Validation Loss: 0.8950962294702944


100%|██████████| 459/459 [02:38<00:00,  2.90it/s]


Epoch 3 - Training Loss: 0.8855408537232019 Validation Loss: 0.8957864465920822


100%|██████████| 459/459 [02:39<00:00,  2.88it/s]


Epoch 4 - Training Loss: 0.8488008047500727 Validation Loss: 0.7191542037155317


In [75]:
sample[0]

tensor([[  101,  1789,  1104,  ...,     0,     0,     0],
        [  101,  1109,  1503,  ...,     0,     0,     0],
        [  101,   157, 20240,  ...,     0,     0,     0],
        ...,
        [  101,  1370,  1122,  ...,     0,     0,     0],
        [  101,  1109,  2890,  ...,     0,     0,     0],
        [  101,  1109,  4180,  ...,     0,     0,     0]])

In [82]:
print(model(sample[0].to(device), sample[1].to(device)))
print(sample[2])

tensor([[-0.8886, -0.9723],
        [ 1.3901,  1.4728],
        [-1.0002, -1.0053],
        [ 0.3164,  0.0920],
        [-0.7886, -0.8047],
        [-0.4184, -0.5556],
        [ 1.3233,  0.7701],
        [-0.3682, -0.6701],
        [-0.5102, -0.5028],
        [ 0.1370,  0.6021]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-1.5472, -1.4612],
        [ 1.1083,  1.6282],
        [-0.7147, -0.9129],
        [ 0.5591, -0.6349],
        [-1.5472, -1.4612],
        [-0.6119, -0.1629],
        [ 1.1138,  0.4169],
        [-0.3823, -1.7955],
        [-0.1732, -0.1285],
        [ 1.0487,  1.2929]])
